## References
- [Convolutional Neural Network for Text Classification in Tensorflow](https://github.com/dennybritz/cnn-text-classification-tf) by dennybritz
- [WILDML - IMPLEMENTING A CNN FOR TEXT CLASSIFICATION IN TENSORFLOW](http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/)
- [WILDML - UNDERSTANDING CONVOLUTIONAL NEURAL NETWORKS FOR NLP](http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/)


In [1]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
import numpy as np

import datetime
import os
import re
import time

## Data

In [2]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


def load_data_and_labels(positive_data_file, negative_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    positive_examples = list(open(positive_data_file, "r").readlines())
    positive_examples = [s.strip() for s in positive_examples]
    negative_examples = list(open(negative_data_file, "r").readlines())
    negative_examples = [s.strip() for s in negative_examples]
    
    # Split by words
    x_text = positive_examples + negative_examples
    x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    positive_labels = [[0, 1] for _ in positive_examples]
    negative_labels = [[1, 0] for _ in negative_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return [x_text, y]

In [3]:
pos_path = "../data/rt-polaritydata/rt-polarity.pos"
neg_path = "../data/rt-polaritydata/rt-polarity.neg"
train_count = 8500

x_text, y = load_data_and_labels(pos_path, neg_path)

In [4]:
# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])
# VocabularyProcessor -> Maps documents to sequences of word ids.
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))

# Random shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

# Split train/test set
train_X, train_y = x_shuffled[:train_count], y_shuffled[:train_count]
dev_X, dev_y = x_shuffled[train_count:], y_shuffled[train_count:]

print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(train_y), len(dev_y)))

Vocabulary Size: 18758
Train/Dev split: 8500/2162


In [5]:
print("Train total count : " + str(len(train_X)))
print("Sequence Length: " + str(train_X.shape[1]))
train_X[:1]

Train total count : 8500
Sequence Length: 56


array([[4719,   59,  182,   34,  190,  804,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0]])

## Model

In [6]:
class TextCNN(object):
    
    def __init__(self, sequence_length, num_classes, vocab_size,
                 embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):
        
        # Input Placeholder
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        
        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)
        
        # Embedding Layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            W = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0), name="W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)
            
        # Convolution & Max-Pooling Layers for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded, 
                    W, 
                    strides=[1,1,1,1], 
                    padding="VALID", 
                    name="conv"
                )
                
                # Apply Relu
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                
                # Max-pooling
                pooled = tf.nn.max_pool(
                    h, 
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1,1,1,1],
                    padding='VALID',
                    name="pool"
                )
                pooled_outputs.append(pooled)
                
        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        
        # Dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)
            
        # Score and Predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer()
            )
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")
            
        # Loss and Accuracy - Calcuate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + (l2_reg_lambda * l2_loss)
            
        # Calculate Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float", name="accuracy"))            

## Training

In [7]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int(len(data)/batch_size)
    print(num_batches_per_epoch)
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]
            
model_config = {
    "sequence_length": train_X.shape[1],
    "num_classes": 2,
    "vocab_size": len(vocab_processor.vocabulary_),
    "embedding_size": 128,
    "filter_sizes": [3, 4, 5],
    "num_filters": 128
}

train_config = {
    "dropout_keep_prob": 0.5,
    "learning_rate": 1e-4,
    "batch_size": 64,
    "num_epochs": 150,
    "num_checkpoints": 5,
    "evaluate_every": 1000,
    "checkpoint_every": 1000,
    "verbose_step": 100
}

In [8]:
def train():
    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=True,
          log_device_placement=False
        )
        sess = tf.Session(config=session_conf)

        with sess.as_default():
            text_cnn = TextCNN(**model_config)

            # Define Training procedure Adam optiminzer
            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.train.AdamOptimizer(train_config["learning_rate"])
            grads_and_vars = optimizer.compute_gradients(text_cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Keep track of gradient values and sparsity (optional)
            grad_summaries = []
            for g, v in grads_and_vars:
                if g is not None:
                    grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                    sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                    grad_summaries.append(grad_hist_summary)
                    grad_summaries.append(sparsity_summary)
            grad_summaries_merged = tf.summary.merge(grad_summaries)

            # Output directory for models and summaries
            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
            print("Writing to {}\n".format(out_dir))

            # Summaries for loss and accuracy
            loss_summary = tf.summary.scalar("loss", text_cnn.loss)
            acc_summary = tf.summary.scalar("accuracy", text_cnn.accuracy)

            # Summary - TensorBoard
            train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
            dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
            dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)
            
            # Checkpointing - Tensorflow assumes thks directory already exists so we need to create it
            out_dir = "text_cnn"
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=train_config["num_checkpoints"])

            # Init
            sess.run(tf.global_variables_initializer())

            # train_step
            def train_step(batch_x, batch_y):
                # A Single training step
                feed_dict = {
                    text_cnn.input_x: batch_x,
                    text_cnn.input_y: batch_y,
                    text_cnn.dropout_keep_prob: train_config["dropout_keep_prob"]
                }
                _, step, summaries, loss, accuracy = sess.run(
                    [train_op, global_step, train_summary_op, text_cnn.loss, text_cnn.accuracy],
                    feed_dict
                )

                time_str = datetime.datetime.now().isoformat()
                if step % train_config["verbose_step"] == 0:
                    print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))

            def dev_step(batch_x, batch_y, writer=None):
                # Evaluates model on a dev set
                feed_dict = {
                    text_cnn.input_x: batch_x,
                    text_cnn.input_y: batch_y,
                    text_cnn.dropout_keep_prob: 1.0
                }
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, text_cnn.loss, text_cnn.accuracy],
                    feed_dict
                )
                
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                if writer:
                    writer.add_summary(summaries, step)
                
            # Generate batches
            batches = batch_iter(
                list(zip(train_X, train_y)), train_config["batch_size"], train_config["num_epochs"])

            # Trainig loop. For each batch...
            for batch in batches:
                x_batch, y_batch = zip(*batch)
                train_step(x_batch, y_batch)
                current_step = tf.train.global_step(sess, global_step)
                
                if current_step % train_config["evaluate_every"] == 0:
                    print("\nEvalutaion:")
                    dev_step(dev_X, dev_y, writer=dev_summary_writer)
                
                if current_step % train_config["checkpoint_every"] == 0:
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(checkpoint_prefix + "-" + str(current_step)))
#                     print("Saved model checkpoint to {}\n".format(path))

In [9]:
train()

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; using 

## Evalutaion

In [16]:
raw_x = ["a masterpiece four years in the making", "everything is off."]

test_X = x_shuffled
test_y = y_shuffled

eval_config = {
    "batch_size": 64,
    "checkpoint_dir": "./text_cnn/checkpoints/",
    "eval_train": False
}

In [17]:
def evalutaion():
    checkpoint_file = tf.train.latest_checkpoint(eval_config["checkpoint_dir"])
    graph = tf.Graph()
    with graph.as_default():
        session_conf = tf.ConfigProto(
            allow_soft_placement=True,
            log_device_placement=False
        )
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            # Load the saved meta graph and restore variables
            saver = tf.train.import_meta_graph(str(checkpoint_file) + ".meta")
            saver.restore(sess, checkpoint_file)
            
            # Get the placeholders from the graph by name
            input_x = graph.get_operation_by_name("input_x").outputs[0]
            input_y = graph.get_operation_by_name("input_y").outputs[0]
            dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]
            
            # Tensors we want to evalutae
            predictions = graph.get_operation_by_name("output/predictions").outputs[0]
            
            # Generate batches for one epoch
            batches = batch_iter(test_X, eval_config["batch_size"], 1, shuffle=False)
            
            # Collect the predictions here
            all_predictions = []
            
            for batch_test_x in batches:
                batch_predictions = sess.run(
                    predictions,
                    {input_x: batch_test_x, dropout_keep_prob: 1.0}
                )
                all_predictions = np.concatenate([all_predictions, batch_predictions])
                
    # Print accuracy if test_y is defind
    if test_y is not None:
        correct_predictions = 0
        for prediction, y in zip(all_predictions, test_y):
            if y[int(prediction)] == 1:
                correct_predictions += 1 
        print("Total number of test examples: {}".format(len(test_y)))
        print("Accuracy: {:g}".format(correct_predictions/float(len(test_y))))

In [18]:
evalutaion()

166
Total number of test examples: 10662
Accuracy: 0.946164
